In [1]:
import pandas as pd
import requests
from urllib.parse import urljoin
import json

In [168]:
df = pd.read_excel('all_afam_latino_with_ms_in_ucsf_apex.xlsx', dtype={'PAT_MRN_ID': str})

auth_info = {
    'userId': 'phamlinh',
    'password': 'Uc$fpride2023'
}

radURL = "https://air.radiology.ucsf.edu/api/"
# Initialize AIR session and store authorization token
session = requests.post(urljoin(radURL, 'login'), json = auth_info).json()

jwt = session['token']['jwt']
header = {'Authorization': 'Bearer ' + jwt}

In [55]:
df.head()

,PAT_MRN_ID,Sex,Birthdate,First Race,Race,ETHNICITY,DX_NAME,DiseaseDuration,Recent DMT,DMT Order Date,Latest EDSS Date,LatestEDSS Value
0,22453181,Female,1959-07-14,Black or African American,Black or African American,Not Hispanic or Latino,NaN,NaN,NaN,NaT,NaT,NaN
1,20313161,Male,1959-07-23,Black or African American,Black or African American,Not Hispanic or Latino,NaN,NaN,NaN,NaT,NaT,NaN
2,11884733,Female,1954-08-05,White,White,Hispanic or Latino,"Multiple sclerosis, secondary progressive (CMS...",0.0,siponimod,2023-02-21,2020-05-11,7.0
3,49262892,Female,1984-12-29,Black or African American,Black or African American,Not Hispanic or Latino,Multiple sclerosis (CMS code),9.0,natalizumab,2014-12-17,2019-04-02,1.5
4,43479989,Female,1973-01-26,Other,Other,Hispanic or Latino,Multiple sclerosis (CMS code),8.0,dimethyl fumarate,2017-05-01,2019-02-05,4.5


In [247]:
def query_accessions_from_mrn(mrn):
    search_json = {
            'accNum': "",
            'mrn': mrn,
            'dateRange': {'start': '', 'end': ''},
            'modality': "MR",
            'name': "",
            'sourceId':1,
            'studyDescription':"",
            'studyUid':""
    }
    exam_lst = requests.post(urljoin(radURL, 'secure/search/query-data-source'), 
                            headers = header,
                            json = search_json).json()['exams'] 
            
    return exam_lst

In [82]:
dfr = df.head(2).PAT_MRN_ID.apply(query_accessions_from_mrn)

In [149]:
dfr.iloc[0]#.explode()#.iloc[0]

[{'patientName': 'HILL, JERI',
  'patientId': '22453181',
  'issuerOfPatientId': 'UCSF',
  'sex': 'F',
  'birthdate': '1959-07-14T00:00:00.000-07:00',
  'accessionNumber': '10141690',
  'description': 'MR SHOULDER WITHOUT CONTRAST, LEFT',
  'modality': 'MR',
  'dateTime': '2014-11-19T13:15:13.000-08:00',
  'studyUid': '1.2.124.113532.80.22017.45499.20141110.123329.78733591',
  'imageCount': 179,
  'deviceId': 9,
  'online': True,
  'viewable': True,
  'devicePriority': 6},
 {'patientName': 'HILL, JERI',
  'patientId': '22453181',
  'issuerOfPatientId': 'UCSF',
  'sex': 'F',
  'birthdate': '1959-07-14T00:00:00.000-07:00',
  'accessionNumber': '13318432',
  'description': 'MR BRAIN WITH AND WITHOUT CONTRAST',
  'modality': 'MR',
  'dateTime': '2019-12-03T16:01:08.000-08:00',
  'studyUid': '1.2.124.113532.80.22185.43466.20191101.144334.139409260',
  'imageCount': 1946,
  'deviceId': 9,
  'online': True,
  'viewable': True,
  'devicePriority': 6},
 {'patientName': 'HILL, JERI',
  'patientI

In [122]:
dfr.explode().to_list()
#pd.DataFrame.from_records(dfr.explode().to_list())

[{'patientName': 'HILL, JERI',
  'patientId': '22453181',
  'issuerOfPatientId': 'UCSF',
  'sex': 'F',
  'birthdate': '1959-07-14T00:00:00.000-07:00',
  'accessionNumber': '10141690',
  'description': 'MR SHOULDER WITHOUT CONTRAST, LEFT',
  'modality': 'MR',
  'dateTime': '2014-11-19T13:15:13.000-08:00',
  'studyUid': '1.2.124.113532.80.22017.45499.20141110.123329.78733591',
  'imageCount': 179,
  'deviceId': 9,
  'online': True,
  'viewable': True,
  'devicePriority': 6},
 {'patientName': 'HILL, JERI',
  'patientId': '22453181',
  'issuerOfPatientId': 'UCSF',
  'sex': 'F',
  'birthdate': '1959-07-14T00:00:00.000-07:00',
  'accessionNumber': '13318432',
  'description': 'MR BRAIN WITH AND WITHOUT CONTRAST',
  'modality': 'MR',
  'dateTime': '2019-12-03T16:01:08.000-08:00',
  'studyUid': '1.2.124.113532.80.22185.43466.20191101.144334.139409260',
  'imageCount': 1946,
  'deviceId': 9,
  'online': True,
  'viewable': True,
  'devicePriority': 6},
 {'patientName': 'HILL, JERI',
  'patientI

In [ ]:
import dask.dataframe as dd
from itertools import chain

npartitions = 5
ddf = dd.from_pandas(df.PAT_MRN_ID, npartitions=npartitions)
ddfr = ddf.apply(query_accessions_from_mrn, meta=list).compute().apply(eval)

null_filter = ddfr.apply(len) == 0 # Boolean series to identify subjects without exams

#exam_df = pd.DataFrame.from_records(ddfr.explode().tolist())
exam_df = pd.DataFrame.from_records(chain(ddfr[~null_filter].explode()))


exam_df = exam_df[exam_df.modality == 'MR']

In [241]:
exam_counts#.loc['22453181']

patientId
61903976    60
48883940    59
10088194    57
44674874    47
47954070    47
            ..
79790320     1
50071057     1
32298524     1
68706670     1
67295129     1
Name: count, Length: 1142, dtype: int64

In [251]:
exam_counts = exam_df['patientId'].value_counts().astype(int)
exam_counts.rename('Exam count')
enriched_df = df.join(exam_counts, how='left', on='PAT_MRN_ID')
enriched_df['count'] = enriched_df['count'].fillna(0)


In [253]:
with pd.ExcelWriter('all_afam_latino_with_ms_in_ucsf_apex_with_exam_count.xlsx') as writer:
    enriched_df.to_excel(writer)
    exam_df.to_excel(writer, sheet_name='Exam metadata')